<a href="https://colab.research.google.com/github/nickradunovic/ragdemo_politie/blob/main/rechtspraak_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Augmented Generation in LLMs

[Introductory text]


## CHAPTER 1: Creating the embedded model and vector database

In [ ]:
# First, we make sure to install the required dependencies.
!pip install python-dotenv
!pip install openai
!pip install azure-search-documents
!pip install azure-core

# CHAPTER 2: Create the LLM prompt with question and answer

In [ ]:
import uuid
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import List
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from google.colab import userdata

load_dotenv()

credential = AzureKeyCredential(os.environ.get("SEARCH_KEY"))
search_client = SearchClient(
    endpoint=os.environ.get("SEARCH_ENDPOINT"),
    index_name=os.environ.get("SEARCH_INDEX_NAME"),
    credential=credential,
)
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_ENDPOINT"),
)

In [ ]:
class Assistant:
    def __init__(self, search_client: SearchClient, openai_client):
        self.search_client = search_client
        self.llm = openai_client.chat.completions.create
        self.embedding = openai_client.embeddings.create

    def search(self, query: str, top_n_documents: int) -> List:
        """
        Searches for documents based on the given query.

        Args:
            query (str): The search query.
            top_n_documents (int): Number of top documents to retrieve.

        Returns:
            list: A list of search results.
        """
        query_embedding = (
            self.embedding(input=[query], model=userdata.get("EMBEDDING_NAME"))
            .data[0]
            .embedding
        )

        # Azure AI search requires a vector query
        vector_query = VectorizedQuery(
            vector=query_embedding,
            k_nearest_neighbors=top_n_documents,
            fields="content_vector",
            exhaustive=True,
        )

        # passing in query in search_text makes it 'hybrid' search
        search_results = self.search_client.search(
            search_text=query, vector_queries=[vector_query], top=top_n_documents
        )

        return search_results

    def rag_chat(self, question: str) -> str:
        """
        Ask a question to the RAG chatbot

        Args:
            question (str): The question to ask the chatbot.

        Returns:
            str: The response from the chatbot.
        """
        search_results = self.search(question, top_n_documents=7)

        documents_string = ""
        for result in search_results:
            documents_string += (
                f"ECLI: {result['title']} | Text: {result['content']}\n\n"
            )

        response = self.llm(
            model="gpt-35-turbo",  # gpt-35-turbo
            messages=[
                {
                    "role": "user",
                    "content": f"Je bent een assistent die vragen beantwoordt over Rechtspraken. \
                        Bij elke vraag krijg je relevante info van verschillende rechtszaken meegestuurd in CONTEXT. \
                        De vraag staat bij VRAAG. \
                        Je moet je antwoord enkel en alleen baseren op de meegestuurde info \
                        Refereer in je antwoord per stuk. \
                        VRAAG: {question}. CONTEXT:\n{documents_string}. \nJOUW ANTWOORD:",
                },
            ],
        )

        return response.choices[0].message.content, documents_string

    def normal_chat(self, question: str) -> str:
        """
        Ask a question to the RAG chatbot

        Args:
            question (str): The question to ask the chatbot.

        Returns:
            str: The response from the chatbot.
        """
        response = self.llm(
            model="gpt-35-turbo",  # gpt-35-turbo
            messages=[
                {
                    "role": "system",
                    "content": "Je bent een assistent die vragen beantwoordt over Rechtspraken.",
                },
                {"role": "user", "content": f"VRAAG: {question}"},
            ],
        )

        return response.choices[0].message.content

In [ ]:
assistant = Assistant(search_client, client)

In [ ]:
# vraag = "welke rechtszaken hebben een man als dader"
# vraag = "in welke rechtszaken komt het gebruik van een vuurwapen voor?"
vraag = "In welke rechtzaken zijn kinderen betrokken? Benoem ook de id van de rechtzaak"
rag_antwoord, rag_documents = assistant.rag_chat(vraag)
antwoord, documents = assistant.rag_chat(vraag)

In [ ]:
print(rag_antwoord)
# print(rag_documents)
# print(antwoord)
# print(documents)

In de volgende rechtszaken zijn kinderen betrokken:

- ECLI:NL:RBNHO:2021:9613: In deze zaak zijn twee minderjarigen [kind 1] en [kind 2] betrokken. Ze zijn door de man erkend en hebben hun hoofdverblijfplaats bij de vrouw. Partijen zijn gezamenlijk belast met het gezag over de minderjarigen.

- ECLI:NL:RBROT:2021:10072: In deze zaak zijn drie minderjarigen, genaamd [naam kind 1], [naam kind 2] en [naam kind 3], betrokken. De kinderrechter acht een verlenging van de ondertoezichtstelling voor de duur van negen maanden noodzakelijk. Het is van belang dat op een zo kort mogelijke termijn duidelijkheid komt over het hoofdverblijf van de kinderen en dat afspraken over de kinderalimentatie worden vastgelegd. Met name is het van belang dat een omgangsregeling tussen de moeder en de kinderen wordt vormgegeven - op een manier die voor beide ouders haalbaar zal zijn en die aansluit op de behoeften van de kinderen.

- ECLI:NL:GHARL:2020:219: In deze zaak zijn [de minderjarige1] en [de minderjari